In [1]:
import os 
import pandas as pd
import numpy as np
import subprocess
import sys
import shutil
from distutils.dir_util import copy_tree
import distutils
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
import timeit


%matplotlib notebook

### Local Functions

In [ ]:
def TS_Describe_stats(data, valType):
    mindate = data['datetime'].min()
    maxdate = data['datetime'].max()
    daterez = data['datetime'][1]-data['datetime'][0]
    minval = data[valType].min()
    maxval = data[valType].max()
    aveval = data[valType].mean()
    medval = data[valType].median()
    print("Date range = {} to {}".format(mindate, maxdate))
    print("Timestep is {}".format(daterez))
    print("Value range is {} to {}".format(minval, maxval))   
    print("Average value is {}".format(aveval))
    print("Median value is {}".format(medval)) 
    print('\n')
    
    
### The rain gage file maker function for a single rain gauge ### 
# Input_Precip_df - A dataframe of timerseries precipitation data 
# Precip_column_Name - The column in the above Input_Precip_df to use
# gagFileFolder-  The relative path to the gag file formatted as 
# StartDate - Desired start date of the run (note will work if no hours or minutes but best to add them)
# EndDate  - Desired end date of the run 
# Lat lon of the gauge site, important for GSSHA calculations
# GageName - name of gauge, not so important 
# RainSeries_timestep_Mins - The timestep of the input rainfall data from Input_Precip_df
# ImpPrecip_units - Units of the precip df, this converts it to mm before running in GSSHA as mm is the default GSSHA units
    
def make_rain_gag_file(PrjName, Input_Precip_df, Precip_column_Name, gagFileFolder, StartDate, EndDate, Lat, Lon, 
                       GageName, RainSeries_timestep_Mins, ImpPrecip_units="Inches"):

    Full_Frame = Input_Precip_df   # read in data 
    SliceFrame = Full_Frame[StartDate:EndDate]

    # Format date string for dumb file 
    SliceFrame_format = SliceFrame.copy()
    SliceFrame_format['da'] = SliceFrame_format.index.strftime('%Y %m %d %H %M')
    SliceFrame_format['datedumb'] = SliceFrame_format['da'].str[:]

    # Pull extranious columns 
    SliceFrame_format = SliceFrame_format[['datedumb', Precip_column_Name]]

    # Turn rain in to rain MM 
    if ImpPrecip_units == "Inches": 
        SliceFrame_format[Precip_column_Name+"_mm"] = SliceFrame_format[Precip_column_Name]*25.4 
        Precip_column_Name = Precip_column_Name+"_mm"
        
    # Round off the number of significant figs
    sigfigs = 3
    SliceFrame_format[Precip_column_Name] = SliceFrame_format[Precip_column_Name].round(sigfigs).apply(lambda x: 
                                                                                                 '{0:g}'.format(float(x)))
    # Put the GAGES card on EVery single row 
    SliceFrame_format["trash"] = "GAGES"

    # reorder columns 
    SliceFrame_format = SliceFrame_format[["trash", "datedumb", Precip_column_Name]]

    # Print it off to a txt file with no header, no index and space separator
    FileNamePlace = os.path.join(gagFileFolder, "{}.gag".format(PrjName))
    SliceFrame_format.to_csv(FileNamePlace, index=False, sep=' ', header = False) 

    # Remove dumb double quotes 
    with open(FileNamePlace,'r') as file:
        data = file.read()
        data = data.replace('"','')
    with open(FileNamePlace,'w') as file:    
        file.write(data)

    # Create the required header lines for WMS gag  files 
    AddLine = 'EVENT "Rain Gage" \nNRGAG 1 \nNRPDS {}\nCOORD {} {} "{}"'.format(len(SliceFrame_format),Lat, Lon, GageName)  

    with open(FileNamePlace, "r+") as f:
        old = f.read() # read everything in the file
        f.seek(0) # rewind
        f.write("{}\n".format(AddLine) + old) # write the new line before

    # Calculate the number of minutes to run the damn thing. 
    Run_Length_mins = RainSeries_timestep_Mins*len(SliceFrame_format)
    print("Simulation time is {} minutes".format(Run_Length_mins))
    print("number of time rows is {}".format(len(SliceFrame_format)))
    
    return Run_Length_mins, SliceFrame_format




#### Process outlet file from GSSHA into a pandas dataframe ###
# StartDate - has to have hours and minutes e.g. 00:00
# OutletFile  - the gssha .otl file from the project 

def process_otl_file(StartDate, OutletFile):

    # Create the start time object to enumerate the number of minutes in the outlet file 
    StartDateTime = datetime.strptime(StartDate, '%Y-%m-%d %H:%M')
    
    # read in the outlet file 
    OutHydro = pd.read_csv(OutletFile, names=["Minutes", "CFS"], delim_whitespace=True)
    
    # da magic: Turn stupid minutes into useful datetime objects 
    OutHydro["datetime"] = OutHydro["Minutes"].apply(lambda x: StartDateTime + timedelta(minutes=x))
    
    # Set the index to the date
    OutHydro.set_index("datetime", inplace=True)
    
    return OutHydro



#### Process streamflow datasets from da streamflow dataframe  ### 

def Isolate_Stream_Data(Input_Stream_df, StreamFlow_column_Name, StartDate, EndDate, 
                        StreamFlowObs_resample_Timestep_mins = 60):
    SliceFrame = Input_Stream_df[StartDate:EndDate]
    
    # USGS data is amazingly terrible, it does not have consistent timesteps, fix this issue here
    mins = StreamFlowObs_resample_Timestep_mins
    SliceFrame = SliceFrame.resample('{}T'.format(mins)).mean().interpolate()

    return SliceFrame[[StreamFlow_column_Name]]



### Calculate NSE of predictions and obs  #### 
def nse(predictions, targets):
    return (1-(np.sum((predictions-targets)**2)/np.sum((targets-np.mean(targets))**2)))


#### Open up the parameters file and replace certain keys with certain values   #### 
def cmt_prama_jama(MapTableFile, Param, Val):
    with open(MapTableFile, 'r') as file :    # Read in the file 
        filedata = file.read()

    filedata = filedata.replace(Param, Val)  # Replace the target paramater(s)

    with open(MapTableFile, 'w') as file:   # Write the file out again
        file.write(filedata)

In [ ]:
####### RUN THIS CELL ONLY WHEN SAVING A FRESH PROJECT FILE FROM WMS in the RUN directory!

#### Nuke out the PRISTINE_MODEL_COPY directory to start fresh
#for f in os.listdir("PRISTINE_MODEL_COPY"):  os.remove(os.path.join("PRISTINE_MODEL_COPY", f))

#### This will copy out the project files to the working directory  
#copy_tree("RUN", 'PRISTINE_MODEL_COPY')

#### Copy in GSSHA exe
#distutils.file_util.copy_file(os.path.join(".", "data", "gssha.exe"), 'PRISTINE_MODEL_COPY')

#### Nuke out the RUN directory to start fresh
#for f in os.listdir("RUN"):  os.remove(os.path.join("RUN", f))

# Rainfall data 

In [ ]:
# Import the three closest rainfall stations into a rainfall file 

# Import KAHAKULOA station 
KAHAKULOARain = pd.read_csv(os.path.join(".", "data/precip", 'KHKH1.csv'))
KAHAKULOARain['DateTime'] = pd.to_datetime(KAHAKULOARain['DateTime'], errors='coerce')
# Clean data 
KAHAKULOARain = KAHAKULOARain[['DateTime', 'RF_mm']]
KAHAKULOARain.rename(columns={'DateTime': 'datetime'}, inplace=True)
# Plot data 
fig, ax = plt.subplots(figsize=(6, 2))
ax.plot(KAHAKULOARain['datetime'],KAHAKULOARain['RF_mm'], '-',label='Rainfall Inches')
TS_Describe_stats(KAHAKULOARain, 'RF_mm')

# Import PUU_KUKIv2 station 
PUU_KUKIv2 = pd.read_csv(os.path.join(".", "data/precip", 'USGS_uv205327156351102.csv'))
PUU_KUKIv2['DateTime'] = pd.to_datetime(PUU_KUKIv2['DateTime'], errors='coerce')
# Clean data 
PUU_KUKIv2 = PUU_KUKIv2[['DateTime', 'RF_mm']]
PUU_KUKIv2.rename(columns={'DateTime': 'datetime'}, inplace=True)
# Plot data 
fig, ax = plt.subplots(figsize=(6, 2))
ax.plot(PUU_KUKIv2['datetime'],PUU_KUKIv2['RF_mm'], '-',label='Rainfall Inches')
TS_Describe_stats(PUU_KUKIv2, 'RF_mm')

# Import WAILUKU_Rain station 
WAILUKU_Rain = pd.read_csv(os.path.join("..", "..", "Data/Rainfall_data/Yufen_paper", 'WUKH1.csv'))
WAILUKU_Rain['DateTime'] = pd.to_datetime(WAILUKU_Rain['DateTime'], errors='coerce')
# Clean data 
WAILUKU_Rain = WAILUKU_Rain[['DateTime', 'RF_mm']]
WAILUKU_Rain.rename(columns={'DateTime': 'datetime'}, inplace=True)
# Plot data 
fig, ax = plt.subplots(figsize=(6, 2))
ax.plot(WAILUKU_Rain['datetime'],WAILUKU_Rain['RF_mm'], '-',label='Rainfall Inches')
TS_Describe_stats(WAILUKU_Rain, 'RF_mm')

In [ ]:
# DONT Resample rainfall, (it makes the nans into 0s)combine 
KAHAKULOARain_set = KAHAKULOARain.set_index('datetime')
KAHAKULOARain_set.rename(columns={'RF_mm': 'KAHAKULOARain'}, inplace=True)

PUU_KUKIv2_set = PUU_KUKIv2.set_index('datetime')
PUU_KUKIv2_set.rename(columns={'RF_mm': 'PUU_KUKIv2'}, inplace=True)

WAILUKU_Rain_set = WAILUKU_Rain.set_index('datetime')
WAILUKU_Rain_set.rename(columns={'RF_mm': 'WAILUKU_Rain'}, inplace=True)

# Merge themm all togehter
m1 = PUU_KUKIv2_set.join(KAHAKULOARain_set, how='outer')
Merged_Rainfall_mm = m1.join(WAILUKU_Rain_set)

# Create an average column of the three
Merged_Rainfall_mm['MEAN_all']= Merged_Rainfall_mm.mean(axis=1)


# Plot em all 
fig, ax = plt.subplots(figsize=(10, 4))
ax.plot(Merged_Rainfall_mm['PUU_KUKIv2'], '-', c='orange', alpha=0.5, label="PUU_KUKIv2")
ax.plot(Merged_Rainfall_mm['KAHAKULOARain'], '-', c='b', alpha=0.5, label="KAHAKULOARain")
ax.plot(Merged_Rainfall_mm['WAILUKU_Rain'], '-', c='pink', alpha=0.5, label="WAILUKU_Rain")
ax.plot(Merged_Rainfall_mm['MEAN_all'], ':', c='k', alpha=0.2, label="MEAN_all")

plt.legend()

# Process Stream files 

Note that this leaves them at different time steps resolutions
Will need to resample to the GSSHA output timesep when using them for comparisons

In [ ]:
# Import Wailuku data 
WailukuFlow = pd.read_csv(os.path.join(".", "data/Streamflow", 'FlowWailukuKepaniwai_2000-01-01_to_2023-05-01.csv'))
WailukuFlow['value_time'] = pd.to_datetime(WailukuFlow['value_time'], errors='coerce')
# Clean data 
WailukuFlow = WailukuFlow[['value_time', 'value']]
WailukuFlow.rename(columns={'value': 'WailukuFlow', 'value_time':'datetime'}, inplace=True)
# Plot data 
fig, ax = plt.subplots(figsize=(6, 2))
ax.plot(WailukuFlow['datetime'],WailukuFlow['WailukuFlow'], '-',label='WailukuFlow')
TS_Describe_stats(WailukuFlow, "WailukuFlow")
WailukuFlow = WailukuFlow.set_index('datetime')  # set index to datetime index for use later 


# Import Waihehe data 
WaiheheFlow = pd.read_csv(os.path.join(".", "data/Streamflow", 'FlowWaihehe_2000-01-01_to_2023-05-01.csv'))
WaiheheFlow['value_time'] = pd.to_datetime(WaiheheFlow['value_time'], errors='coerce')
# Clean data 
WaiheheFlow = WaiheheFlow[['value_time', 'value']]
WaiheheFlow.rename(columns={'value': 'WaiheheFlow', 'value_time':'datetime'}, inplace=True)
# Plot data 
fig, ax = plt.subplots(figsize=(6, 2))
ax.plot(WaiheheFlow['datetime'],WaiheheFlow['WaiheheFlow'], '-',label='WaiheheFlow')
WaiheheFlow = WaiheheFlow.set_index('datetime')  # set index to datetime index for use later 


# Import SWaiehu data   ( note that this file was downloaded manually the API didnt work so the import is a little different than above)
SWaiehuFlow = pd.read_csv(os.path.join(".", "data/Streamflow", 'Flow_SWaiehu_2022_to_2023.csv'))
SWaiehuFlow['value_time'] = pd.to_datetime(SWaiehuFlow['value_time'], errors='coerce')
# Thre were some strings in the value column, freakin USGS....
SWaiehuFlow['value'] = SWaiehuFlow['value'].apply(lambda x: pd.to_numeric(x, errors='coerce') )
# Clean data 
SWaiehuFlow = SWaiehuFlow[['value_time', 'value']]
SWaiehuFlow.rename(columns={'value': 'SWaiehuFlow', 'value_time':'datetime'}, inplace=True)
# Plot data 
fig, ax = plt.subplots(figsize=(6, 2))
ax.plot(SWaiehuFlow['datetime'],SWaiehuFlow['SWaiehuFlow'], '-',label='SWaiehuFlow')
TS_Describe_stats(SWaiehuFlow, 'SWaiehuFlow')
SWaiehuFlow = SWaiehuFlow.set_index('datetime')  # set index to datetime index for use later 


# Mossta loop

In [ ]:
# Get set up to iterate over and run the thing 

# Static parameters for rain function 
Input_Precip_df = Merged_Rainfall_mm
Precip_column_Name = "PUU_KUKIv2"
ImpPrecip_units = "mm"
gagFileFolder = os.path.join(".", "RUN")
PrjName = "Iao_v1_PrecipGag_50m"
Lat = "753250.0" 
Lon = "2310140.0"
GageName = "gag_face"
RainSeries_timestep_Mins = 60

# Static parameters for stream obs function
Input_Stream_df = WailukuFlow
StreamFlow_column_Name = "WailukuFlow"
StreamFlowObs_resample_Timestep_mins = 60   # note this will also be used to change HYD_FREQ in .prj file 

In [ ]:
# Save lists 
Param_Vals_list = []
run_time_list =[]
NSE_list = []








# Modify roughness parameters 
rough_vals_list = ["0.05", "0.1", "0.2", "0.3", "0.4", "0.5", "0.6", "0.7"]

# File where the roughness parameters are defined
MapTableFile = os.path.join(".", "RUN", "{}.cmt".format(PrjName))



# List of parameters in the model
Param_list = ["-107.000000", 
              "-108.000000", 
              "-1014.000000", 
              "-1015.000000", 
              "-1016.000000", 
              "-1017.000000", 
              "-1021.000000", 
              "-1022.000000"]

for i in rough_vals_list:

    # Switch out each parameter with the Uniform value 
    for Param in Param_list:
        cmt_prama_jama(MapTableFile, Param, i)
        
### Note even if not modifying these parameters need to run ALL code used to create model after the WMS save   
### Since its all erased at the end of the block 

    # rain gauge maker code block ready to loop over start and end dates

    StartDate = "2018-02-17 00:00"
    EndDate = "2018-02-21 00:00"

    Run_Length_mins, SlicedRainFrame = make_rain_gag_file(PrjName, Input_Precip_df, Precip_column_Name, gagFileFolder, StartDate, EndDate, Lat, Lon, 
                           GageName, RainSeries_timestep_Mins, ImpPrecip_units)

    # Modify the .prj file to change the TOT_TIME card to whatever the total number of minutes output by the raingag function outputs
    # read in the .prj file as a 1 column csv
    df = pd.read_csv(os.path.join(".", "RUN", "{}.prj".format(PrjName)), names=["moo"] )  # note names=moo is to make a column that will then get chopped off by numpy savetxt

    # ID row of and change the TOT_TIME variable
    singleCol = df.columns[0]                # this is the name of the single column
    idx_tottime = df.loc[df[singleCol].str.contains("TOT_TIME", case=False)].index[0]  # this identifies the index of the TOT_TIME card
    # Set the total time value to something else. 
    df.loc[idx_tottime] = "TOT_TIME      {}".format(Run_Length_mins)

    # ID row of and change the HYD_FREQ variable
    singleCol = df.columns[0]                # this is the name of the single column
    idx_HYD_FREQ = df.loc[df[singleCol].str.contains("HYD_FREQ", case=False)].index[0]  # this identifies the index of the HYD_FREQ card
    # Set the total time value to something else. 
    df.loc[idx_HYD_FREQ] = "HYD_FREQ      {}".format(StreamFlowObs_resample_Timestep_mins)

    # Save the df back to a prj file (the np formulation seems to write better than the pd to csv one)
    np.savetxt(os.path.join(".", "RUN", "{}.prj".format(PrjName)), df.values, fmt="%s")
        
        
        
 
        
        

    ##########    RUN THE MODEL   #################
    # Run the GSSHA process in the command line 
    start_time = timeit.default_timer()     ###### Timer function start

    os.chdir(os.path.join(".", "RUN"))         # Chanfge into run directoy 
    subprocess.call('gssha.exe {}.prj'.format(PrjName), shell=True)
    os.chdir(os.path.join(".."))                # Change back out of run directory 

    elapsed = timeit.default_timer() - start_time        ###### Timer function End, now time in seconds lives in elapsed
    
    
    
    # Postprocessing block 
    # Process the Run's Output hydrograph into a dataframe 
    OutletFile = os.path.join(".", "RUN", PrjName)+".otl"
    OutHydro = process_otl_file(StartDate, OutletFile)

    # slice the desired streamflow obs to the correct period 
    SlicedStreamflow_df = Isolate_Stream_Data(Input_Stream_df, StreamFlow_column_Name, StartDate, EndDate, 
                                              StreamFlowObs_resample_Timestep_mins)

    # NSE calculation          # Note this has hourly resampling hard coded in, not sure if need to change someday 
    predictions = OutHydro
    targets = SlicedStreamflow_df.resample('60T').mean().interpolate()
    NSE_Frame = predictions.join(targets, how='inner')
    NSE_stat = nse(NSE_Frame['CFS'].values, NSE_Frame[StreamFlow_column_Name].values)
    print("The NSE is {}".format(NSE_stat))


    # plot em 
    # For some reason need to cast the rainfall as a number not object?
    SlicedRainFrame[Precip_column_Name] = SlicedRainFrame[Precip_column_Name].apply(lambda x: float(x)) 

    fig, ax = plt.subplots(figsize=(8, 3))
    ax2 = ax.twinx()
    lns0 = ax2.plot(SlicedRainFrame[Precip_column_Name], '.', c='b', alpha=0.4, label="Rainfall ")   # Plot rainfall 
    lns1 = ax.plot(SlicedStreamflow_df[StreamFlow_column_Name], '-', c='g', alpha=0.5, label="Observed Waihehe flow CFS") # Plot Observed
    lns2 = ax.plot(OutHydro['CFS'], '-', c='k', alpha=0.5, label="Modeled_CFS")   # plot modeled 
    ax.set_ylabel("Flow CFS")
    ax2.set_ylabel("Rainfall mm", color='b')
    # Wierd stuff for a twinned  axis legend
    lns = lns0+lns1+lns2; labs = [l.get_label() for l in lns]
    ax.legend(lns, labs, loc=0)


    # Save info
    Param_Vals_list.append(i)
    run_time_list.append(elapsed) 
    NSE_list.append(NSE_stat)
    
    
    #### refreshing the model for each parameter run 
    # Nuke out the RUN directory to start fresh
    for f in os.listdir("RUN"):  os.remove(os.path.join("RUN", f))  
    # copy out the project filesfrom pristine to RUN  
    copy_tree('PRISTINE_MODEL_COPY', "RUN") 
    
    



# Development bin 

In [ ]:
# Get set up to iterate over and run the thing 

# Static parameters for rain function 
Input_Precip_df = Merged_Rainfall_mm
Precip_column_Name = "PUU_KUKIv2"
ImpPrecip_units = "mm"
gagFileFolder = os.path.join(".", "RUN")
PrjName = "Iao_v1_PrecipGag_50m"
Lat = "753250.0" 
Lon = "2310140.0"
GageName = "gag_face"
RainSeries_timestep_Mins = 60

# Static parameters for stream obs function
Input_Stream_df = WailukuFlow
StreamFlow_column_Name = "WailukuFlow"
StreamFlowObs_resample_Timestep_mins = 60   # note this will also be used to change HYD_FREQ in .prj file 

In [ ]:
# rain gauge maker code block ready to loop over start and end dates

StartDate = "2018-02-17 00:00"
EndDate = "2018-02-21 00:00"

Run_Length_mins, SlicedRainFrame = make_rain_gag_file(PrjName, Input_Precip_df, Precip_column_Name, gagFileFolder, StartDate, EndDate, Lat, Lon, 
                       GageName, RainSeries_timestep_Mins, ImpPrecip_units)

In [ ]:
# Modify the .prj file to change the TOT_TIME card to whatever the total number of minutes output by the raingag function outputs
# read in the .prj file as a 1 column csv
df = pd.read_csv(os.path.join(".", "RUN", "{}.prj".format(PrjName)), names=["moo"] )  # note names=moo is to make a column that will then get chopped off by numpy savetxt

# ID row of and change the TOT_TIME variable
singleCol = df.columns[0]                # this is the name of the single column
idx_tottime = df.loc[df[singleCol].str.contains("TOT_TIME", case=False)].index[0]  # this identifies the index of the TOT_TIME card
# Set the total time value to something else. 
df.loc[idx_tottime] = "TOT_TIME      {}".format(Run_Length_mins)

# ID row of and change the HYD_FREQ variable
singleCol = df.columns[0]                # this is the name of the single column
idx_HYD_FREQ = df.loc[df[singleCol].str.contains("HYD_FREQ", case=False)].index[0]  # this identifies the index of the HYD_FREQ card
# Set the total time value to something else. 
df.loc[idx_HYD_FREQ] = "HYD_FREQ      {}".format(StreamFlowObs_resample_Timestep_mins)

# Save the df back to a prj file (the np formulation seems to write better than the pd to csv one)
np.savetxt(os.path.join(".", "RUN", "{}.prj".format(PrjName)), df.values, fmt="%s")

In [ ]:
# Modify roughness parameters 

# File where the roughness parameters are defined
MapTableFile = os.path.join(".", "RUN", "{}.cmt".format(PrjName))

# just using a uniform value for now
Uniform_Val = "0.15"

# List of parameters in the model
Param_list = ["-107.000000", 
              "-108.000000", 
              "-1014.000000", 
              "-1015.000000", 
              "-1016.000000", 
              "-1017.000000", 
              "-1021.000000", 
              "-1022.000000"]

# Switch out each parameter with the Uniform value 
for Param in Param_list:
    cmt_prama_jama(MapTableFile, Param, Uniform_Val)

In [ ]:
# Here is an operational model runner code block 

##########    RUN THE MODEL   #################
# Run the GSSHA process in the command line 
start_time = timeit.default_timer()     ###### Timer function start

os.chdir(os.path.join(".", "RUN"))         # Chanfge into run directoy 
subprocess.call('gssha.exe {}.prj'.format(PrjName), shell=True)
os.chdir(os.path.join(".."))                # Change back out of run directory 

elapsed = timeit.default_timer() - start_time        ###### Timer function End, now time in seconds lives in elapsed

In [ ]:
# Postprocessing block 

# Process the Run's Output hydrograph into a dataframe 
OutletFile = os.path.join(".", "RUN", PrjName)+".otl"
OutHydro = process_otl_file(StartDate, OutletFile)

# slice the desired streamflow obs to the correct period 
SlicedStreamflow_df = Isolate_Stream_Data(Input_Stream_df, StreamFlow_column_Name, StartDate, EndDate, 
                                          StreamFlowObs_resample_Timestep_mins)

# NSE calculation          # Note this has hourly resampling hard coded in, not sure if need to change someday 
#predictions = OutHydro['CFS'].to_numpy()
#targets = SlicedStreamflow_df.resample('60T').mean().to_numpy()
#NSE_stat = nse(predictions, np.nan_to_num(targets))
#print("The NSE is {}".format(NSE_stat))

predictions = OutHydro
targets = SlicedStreamflow_df.resample('60T').mean().interpolate()
NSE_Frame = predictions.join(targets, how='inner')
NSE_stat = nse(NSE_Frame['CFS'].values, NSE_Frame[StreamFlow_column_Name].values)
print("The NSE is {}".format(NSE_stat))


# plot em 
# For some reason need to cast the rainfall as a number not object?
SlicedRainFrame[Precip_column_Name] = SlicedRainFrame[Precip_column_Name].apply(lambda x: float(x)) 

fig, ax = plt.subplots(figsize=(8, 3))
ax2 = ax.twinx()
lns0 = ax2.plot(SlicedRainFrame[Precip_column_Name], '.', c='b', alpha=0.4, label="Rainfall ")   # Plot rainfall 
lns1 = ax.plot(SlicedStreamflow_df[StreamFlow_column_Name], '-', c='g', alpha=0.5, label="Observed Waihehe flow CFS") # Plot Observed
lns2 = ax.plot(OutHydro['CFS'], '-', c='k', alpha=0.5, label="Modeled_CFS")   # plot modeled 
ax.set_ylabel("Flow CFS")
ax2.set_ylabel("Rainfall mm", color='b')
# Wierd stuff for a twinned  axis legend
lns = lns0+lns1+lns2; labs = [l.get_label() for l in lns]
ax.legend(lns, labs, loc=0)



rando notes
Note at 60 sec time step = 208 sec 
at 30 sec step = 219 sec
at 15 = 204 sec 
at 5 sec = 244 sec 
at 15 sec but Quiet card = 207 sec 

In [ ]:
run_time_list